In [37]:
n = 100
np.random.seed(0)
x = np.random.rand(n,1)
y = 4 + 2*x + .05*np.random.rand(n,1)
X = np.c_[np.ones(n), x]

In [44]:
b_ols = np.linalg.inv(X.T @ X) @ X.T @ y
b_ols

array([[4.027904  ],
       [1.99682755]])

In [45]:
eta = .05
its = 100
b_gd = np.array([[.0],[.0]])
for it in range(its):
    grad = 2/n * X.T @ (X @ b_gd - y)
    b_gd = b_gd - eta*grad
b_gd

array([[4.01875911],
       [2.01485506]])

In [146]:
epochs = 10
batches = 5

t0, t1 = epochs/10, epochs
def learning_schedule(t):
    return t0 / (t + t1)


b_sgd = np.array([[.0],[.0]])
for epoch in range(epochs):
    bn = int(n/batches)
    X_shuffled, y_shuffled = skl.utils.resample(X, y, replace=False, random_state=epoch)
    for batch in range(batches):
        Xb = X_shuffled[bn*batch:bn*(batch+1)]
        yb = y_shuffled[bn*batch:bn*(batch+1)]
        grad = 2/bn * Xb.T @ (Xb @ b_sgd - yb)
        eta = learning_schedule(epoch*batches + batch)
        b_sgd = b_sgd - eta*grad
print(b_sgd)

[[3.99789401]
 [1.97782246]]


In [213]:
from StochasticGradientDescent import SGD
sgd = SGD(epochs=20, batches=10, learning_schedule='optimal', cost_function='ridge')
beta = sgd.run(X, y, lmd=.1)
print(beta)

[[3.72007746]
 [1.86825833]]


In [215]:
?sgd.run

Signature: sgd.run(X, y, beta0=None, lmd=None)
Docstring: Runs Stochastic Gradient Descent with inputs to cost function X, y, starting point beta0 is set to all-zeros if None, lmd is Ridge's L2 hyperparameter and is used in learning_schedule 'optimal'.
File:      ~/Documents/stud/FYS-STK4155/project2/source/StochasticGradientDescent.py
Type:      method
